In [1]:
# ResNet网络
import numpy as np
np.random.seed(0)
import tensorflow as tf

In [2]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# 残差单元
# 两个3*3的卷积层，每个卷积层后面跟一个批量归一化层和ReLu激活函数
# 如果想要改变输入X的通道数，这样可以和卷积层的输出相加

class Residual(tf.keras.Model):
    def __init__(self, channels, use_1x1conv=False, strides=1, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.conv1 = tf.keras.layers.Conv2D(channels, kernel_size=3, strides=strides, padding='same')
        self.conv2 = tf.keras.layers.Conv2D(channels, kernel_size=3, padding='same') # 这个地方的strides应该不变还为1
        if use_1x1conv:
            self.conv3 = tf.keras.layers.Conv2D(channels, kernel_size=1, strides=strides)
        else:
            self.conv3 = None
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()
    
    def call(self, X):
        Y = tf.keras.activations.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return tf.keras.activations.relu(Y + X)

In [4]:
blk = Residual(3)
#tensorflow input shpe     (n_images, x_shape, y_shape, channels).
#mxnet.gluon.nn.conv_layers    (batch_size, in_channels, height, width) 
X = tf.random.uniform((4, 6, 6 , 3))
blk(X).shape#TensorShape([4, 6, 6, 3])

TensorShape([4, 6, 6, 3])

In [5]:
blk = Residual(6, use_1x1conv=True, strides=2)
blk(X).shape
#TensorShape([4, 3, 3, 6])

TensorShape([4, 3, 3, 6])

In [6]:
# 残差模块 
# 第一个模块的通道数同输入通道数一致。由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。
# 之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

class ResnetBlock(tf.keras.Model):
    def __init__(self, channels, num_residuals, first_block=False, **kwargs):
        super(ResnetBlock, self).__init__(**kwargs)
        self.ListLayers = []
        for i in range(num_residuals):
            if (i==0) and (not first_block):
                # 每个残差模块中需要利用use_1x1conv将输入的通道数进行转换，这样才能和残差映射相加
                self.ListLayers.append(Residual(channels, use_1x1conv=True, strides=2)) 
            else:
                self.ListLayers.append(Residual(channels))
    
    def call(self, X):
        for layer in self.ListLayers.layers:
            X = layer(X)
        return X

In [7]:
# ResNet模型
# 残差模型的前两层和GoogLeNet一样
# 通道数64，7*7 步幅为2的卷积层 + 批量归一化层
# 3*3步幅为2的最大池化层
# 4个残差模块
# 全局平均池化层
# 全连接输出层 tf.keras.activaitons.softmax

class ResNet(tf.keras.Model):
    def __init__(self, num_blocks, **kwargs):
        super(ResNet, self).__init__(**kwargs)
        self.conv = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.Activation('relu')
        self.mp = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')
        self.resnet_block1 = ResnetBlock(64, num_blocks[0], first_block=True)
        self.resnet_block2 = ResnetBlock(128, num_blocks[1])
        self.resnet_block3 = ResnetBlock(256, num_blocks[2])
        self.resnet_block4 = ResnetBlock(512, num_blocks[3])
        self.gap = tf.keras.layers.GlobalAvgPool2D()
        self.fc = tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax)
    
    def call(self, X):
        X = self.conv(X)
        X = self.bn(X)
        X = self.relu(X)
        X = self.mp(X)
        X = self.resnet_block1(X)
        X = self.resnet_block2(X)
        X = self.resnet_block3(X)
        X = self.resnet_block4(X)
        X = self.gap(X)
        X = self.fc(X)
        return X

ResNet = ResNet([2,2,2,2])

In [8]:
X = tf.random.uniform(shape=(1, 224, 224, 1))
for layer in ResNet.layers:
    X = layer(X)
    print(layer.name, 'output_shape:\t', X.shape)

conv2d_5 output_shape:	 (1, 112, 112, 64)
batch_normalization_4 output_shape:	 (1, 112, 112, 64)
activation output_shape:	 (1, 112, 112, 64)
max_pooling2d output_shape:	 (1, 56, 56, 64)
resnet_block output_shape:	 (1, 56, 56, 64)
resnet_block_1 output_shape:	 (1, 28, 28, 128)
resnet_block_2 output_shape:	 (1, 14, 14, 256)
resnet_block_3 output_shape:	 (1, 7, 7, 512)
global_average_pooling2d output_shape:	 (1, 512)
dense output_shape:	 (1, 10)


In [9]:
# 模型训练

(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.reshape((60000, 28, 28, 1)).astype("float") / 255.0
X_test = X_test.reshape((10000, 28, 28, 1)).astype(np.float32) / 255.0

ResNet.compile(optimizer=tf.keras.optimizers.Adam(),
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

history = ResNet.fit(X_train, Y_train,batch_size=64, epochs=5,validation_split=0.2)

Epoch 1/5
750/750 [==============================] - 18s 21ms/step - loss: 0.6854 - accuracy: 0.7715 - val_loss: 0.4501 - val_accuracy: 0.8334
Epoch 2/5
750/750 [==============================] - 15s 20ms/step - loss: 0.3411 - accuracy: 0.8725 - val_loss: 0.3522 - val_accuracy: 0.8687
Epoch 3/5
750/750 [==============================] - 15s 20ms/step - loss: 0.2821 - accuracy: 0.8961 - val_loss: 0.3574 - val_accuracy: 0.8698
Epoch 4/5
750/750 [==============================] - 15s 20ms/step - loss: 0.2658 - accuracy: 0.9015 - val_loss: 0.3474 - val_accuracy: 0.8724
Epoch 5/5
750/750 [==============================] - 15s 20ms/step - loss: 0.2408 - accuracy: 0.9120 - val_loss: 0.2801 - val_accuracy: 0.9009


In [10]:
history

In [ ]:
histor